# 03 – Data Preparation
## AdventureWorks | Capital Operativo

**Propósito**
Preparar la tabla analítica central que permita comparar la distribución
del capital operativo (inventario) con la generación de ventas, como
insumo para el análisis descriptivo posterior.

Esta fase incluye:
- Selección definitiva de campos
- Definición de claves
- Preparación de métricas base

No incluye interpretación ni conclusiones.

In [1]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=127.0.0.1,1433;"
    "DATABASE=AdventureWorks2019;"
    "UID=sa;"
    "PWD=S1nt3lo_SQL_2026!;"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

In [2]:
pd.read_sql("SELECT @@VERSION", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_55726/101866107.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT @@VERSION", conn)


,
0,Microsoft SQL Server 2022 (RTM-CU22) (KB506845...


In [10]:
product = pd.read_sql("""
    SELECT
        ProductID,
        Name AS ProductName,
        ProductSubcategoryID,
        StandardCost
    FROM Production.Product
""", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_55726/2805546223.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql("""


In [4]:
inventory = pd.read_sql("""
    SELECT
        ProductID,
        SUM(Quantity) AS QuantityOnHand
    FROM Production.ProductInventory
    GROUP BY ProductID
""", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_55726/1823384707.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  inventory = pd.read_sql("""


In [5]:
sales = pd.read_sql("""
    SELECT
        ProductID,
        SUM(LineTotal) AS SalesAmount
    FROM Sales.SalesOrderDetail
    GROUP BY ProductID
""", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_55726/1947980953.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales = pd.read_sql("""


In [11]:
subcategory = pd.read_sql("""
    SELECT
        ProductSubcategoryID,
        ProductCategoryID,
        Name AS ProductSubcategory
    FROM Production.ProductSubcategory
""", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_55726/1652337449.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  subcategory = pd.read_sql("""


In [12]:
category = pd.read_sql("""
    SELECT
        ProductCategoryID,
        Name AS ProductCategory
    FROM Production.ProductCategory
""", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_55726/1762667430.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  category = pd.read_sql("""


In [13]:
df = (
    product
    .merge(inventory, on="ProductID", how="left")
    .merge(sales, on="ProductID", how="left")
    .merge(subcategory, on="ProductSubcategoryID", how="left")
    .merge(category, on="ProductCategoryID", how="left")
)

In [14]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductID             504 non-null    int64  
 1   ProductName           504 non-null    object 
 2   ProductSubcategoryID  295 non-null    float64
 3   StandardCost          504 non-null    float64
 4   QuantityOnHand        432 non-null    float64
 5   SalesAmount           266 non-null    float64
 6   ProductCategoryID     295 non-null    float64
 7   ProductSubcategory    295 non-null    object 
 8   ProductCategory       295 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 35.6+ KB


In [15]:
df.head(10)

,ProductID,ProductName,ProductSubcategoryID,StandardCost,QuantityOnHand,SalesAmount,ProductCategoryID,ProductSubcategory,ProductCategory
0,1,Adjustable Race,NaN,0.0,1085.0,NaN,NaN,NaN,NaN
1,2,Bearing Ball,NaN,0.0,1109.0,NaN,NaN,NaN,NaN
2,3,BB Ball Bearing,NaN,0.0,1352.0,NaN,NaN,NaN,NaN
3,4,Headset Ball Bearings,NaN,0.0,1322.0,NaN,NaN,NaN,NaN
4,316,Blade,NaN,0.0,1361.0,NaN,NaN,NaN,NaN
5,317,LL Crankarm,NaN,0.0,593.0,NaN,NaN,NaN,NaN
6,318,ML Crankarm,NaN,0.0,439.0,NaN,NaN,NaN,NaN
7,319,HL Crankarm,NaN,0.0,797.0,NaN,NaN,NaN,NaN
8,320,Chainring Bolts,NaN,0.0,1136.0,NaN,NaN,NaN,NaN
9,321,Chainring Nut,NaN,0.0,1750.0,NaN,NaN,NaN,NaN


In [17]:
df["InventoryValue"] = df["QuantityOnHand"].fillna(0) * df["StandardCost"].fillna(0)
df["SalesAmount"] = df["SalesAmount"].fillna(0)

In [18]:
df = df[[
    "ProductCategory",
    "ProductSubcategory",
    "ProductID",
    "ProductName",
    "QuantityOnHand",
    "StandardCost",
    "InventoryValue",
    "SalesAmount"
]]

In [21]:
df.head(10)

,ProductCategory,ProductSubcategory,ProductID,ProductName,QuantityOnHand,StandardCost,InventoryValue,SalesAmount
0,NaN,NaN,1,Adjustable Race,1085.0,0.0,0.0,0.0
1,NaN,NaN,2,Bearing Ball,1109.0,0.0,0.0,0.0
2,NaN,NaN,3,BB Ball Bearing,1352.0,0.0,0.0,0.0
3,NaN,NaN,4,Headset Ball Bearings,1322.0,0.0,0.0,0.0
4,NaN,NaN,316,Blade,1361.0,0.0,0.0,0.0
5,NaN,NaN,317,LL Crankarm,593.0,0.0,0.0,0.0
6,NaN,NaN,318,ML Crankarm,439.0,0.0,0.0,0.0
7,NaN,NaN,319,HL Crankarm,797.0,0.0,0.0,0.0
8,NaN,NaN,320,Chainring Bolts,1136.0,0.0,0.0,0.0
9,NaN,NaN,321,Chainring Nut,1750.0,0.0,0.0,0.0
